In [ ]:
!opalconfig new minio

## Import Python Module to Create/Upload New Data Instances
### (Persona: Data Engineer)

Use OPAL "kind" arbitrary data model

In [ ]:
import pandas as pd
from opal import kinds
kinds = kinds.load()

parsed_kind = kinds.lookup("tip_parsed")
translated_kind = kinds.lookup("tip_translated")

In [ ]:
from upload_many import publish_instance, delete_by_kind_id

def _delete_old():
    found = parsed_kind.metadata_search("ch10_name", "652200101100441")
    if found:
        parsed_id = found[0][1]
        parsed_kind.delete_instance(parsed_id)
        delete_by_kind_id(parsed_id, error_ok=True)
        t_found = translated_kind.metadata_search("derived_from", parsed_id)
        for t_f in t_found:
            translated_kind.delete_instance(t_f[1])
            delete_by_kind_id(t_f[1], error_ok=True)

## Parse and Translate NASA Chapter 10 MilStd1553 and ARINC429 Data
### (Persona: Data Engineer)

In [ ]:
!mkdir -p ~/parsed_data ~/translated_data
!rm -rf ~/parsed_data/* ~/translated_data/*
!tip_parse /opt/data/telemetry-demo-data/652200101100441.ch10 -o ~/parsed_data -L warn
!tip_translate_1553 ~//parsed_data/652200101100441_MILSTD1553_F1.parquet DTS1553_Synth_Nasa.yaml -o ~/translated_data -L info
!tip_translate_arinc429 ~/parsed_data/652200101100441_ARINC429_F0.parquet DTS429_Synth_NASA.yaml -o ~/translated_data -L warn

## Upload Parsed and Translated Datasets to Network Data Store
### (Persona: Data Engineer)

In [ ]:
import os
def upload_new_data():
    parsed_path = os.path.expanduser("~/parsed_data")
    parsed_meta = parsed_kind.upload("parsed_data", {})
    parsed_id = parsed_meta["instance_id"]
    parsed_kind.strong_validate_instance(parsed_id)
    display(parsed_meta)

    t1553_path = os.path.expanduser("~/translated_data/652200101100441_MILSTD1553_F1_translated")
    t1553_meta = translated_kind.upload(
        t1553_path, 
        {}, 
        parent_instance=parsed_id
    )
    t1553_id = t1553_meta["instance_id"]
    translated_kind.strong_validate_instance(t1553_id)
    display(t1553_meta)

    t429_path = os.path.expanduser("~/translated_data/652200101100441_ARINC429_F0_translated")
    t429_meta = translated_kind.upload(
        t429_path, 
        {},
        parent_instance=parsed_id
    )
    t429_id = t429_meta["instance_id"]
    translated_kind.strong_validate_instance(t429_id)
    display(t429_meta)
    return parsed_id, t1553_id, t429_id
    
try:
    parsed_id, t1553_id, t429_id = upload_new_data()
except:
    print("This data already existed")
    print("Deleting old data...")
    _delete_old()
    print("Uploading new data...")
    parsed_id, t1553_id, t429_id = upload_new_data()

## Segue To OPAL Catalog

1. (Upper left) Select "File" --> "Hub Control Panel"
2. (Hub Control Panel Tab) Select "Services" --> "opal-catalog"
3. (OPAL Catalog Tab) Select "Login with Keycloak"
4. Click "Edit Columns" button and add "CH10 NAME" and "VALIDATED DATETIME"
5. Place cursor in "Search Catalog" field
6. Type "ch10", arrow down to select suggestion "kind_metadata.ch10_name", then press Enter
7. Search Catalog for ch10 name "652200101100441" -- No entry! Data have not been published.
8. Clear search bar and Enter to reset search
9. Return to notebook tab, see "Publish Datasets To Searchable Catalog"

## Publish Datasets to Searchable Catalog
### (Persona: Data Engineer)

In [ ]:
from upload_many import publish_instance, delete_by_kind_id

publish_instance(parsed_id, "tip_parsed")
publish_instance(t1553_id, "tip_translated")
publish_instance(t429_id, "tip_translated")

## View Published Data in Catalog

1. Return to OPAL Catalog tab
2. Place cursor in "Search Catalog" field
3. Type "ch10", arrow down to select suggestion "kind_metadata.ch10_name", then press Enter
4. Cursor is placed in quotes, type "441" to search the relevant portion of "652200101100441", press Enter
5. Notice the relevant ch10 by the "VALIDATED DATETIME" column
6. Clear the search text and Enter
7. Type "type" then down arrow to select "kind_type" parameter, followed by Enter
8. Search for "tip_translated"
9. Narrow search: 
    - place cursor after "tip_translate"
    - hit space bar then select "AND"
    - type "validated" and select "validated_datetime"
    - type today's date "2022-05-17" and Enter
10. Click on links in "Kind ID" column to view metadata

## Read Datasets from Catalog
### (Persona: Analyst)
#### Latitude/Longitude/Altitude (1553)

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({"font.size":18})

df_1553 = translated_kind.read(t1553_id, "NAV")
where_valid = df_1553[df_1553["NAV-0110"] & df_1553["NAV-0111"]]

fig, ax = plt.subplots()
ax = where_valid.plot(
    kind="scatter", title="Aircraft Position (1553)",
    x="NAV-23", y="NAV-21", c="NAV-25", s=1,
    cmap="viridis", figsize=(15, 10), ax=ax)

ax.set_aspect("equal")
ax.set_xlabel("Longitude [deg]")
ax.set_ylabel("Latitude [deg]")
plt.gcf().get_axes()[1].set_ylabel("Altitude [ft]")
None

#### Engine Status (ARINC429) with Altitude (1553)

In [ ]:
df_turbine_speed = translated_kind.read(t429_id, "Engine_Turbine_RPM_40")
df_exhaust_temp = translated_kind.read(t429_id, "Exhaust_Gas_Temperature_40")
ax = df_turbine_speed.plot(kind="scatter", x="time", y="N2_RPM", figsize=(13, 10), c='k', label="Turbine Speed [RPM, percent max.]")
ax.set_ylabel(r'Turbine Speed AND Exhaust Temp', c='k')
ax.tick_params(axis='y', colors='k')
df_exhaust_temp.plot(kind="line", x="time", y="Exhaust_Gas_Temperature", ax=ax, c='k', label="Exhaust Temp [deg C]")
ax2 = where_valid.plot(kind="line", x="time", y="NAV-25", secondary_y=True, c='b', ax=ax)
ax2.set_ylabel("Altitude [ft]", c='b')
ax2.yaxis.label.set_color('blue')
ax2.tick_params(axis='y', colors='blue')
ax.set_xlabel("Time [Epoch, ns]")
ax.legend()
title = ax.set_title("Engine Data (ARINC429) as Function of Altitude (1553)")

## Delete Data for Next Run

In [ ]:
_delete_old()

### Delete by Catalog Search (Error Case)

In [ ]:
# parsed_id = 'f7966622-1702-477a-afdd-278919a02643'
# parsed_kind.delete_instance(parsed_id)
# delete_by_kind_id(parsed_id)
# t1553_id = '983bb5a7-3da2-4a15-8683-7a7c23cfc3ac'
# translated_kind.delete_instance(t1553_id)
# delete_by_kind_id(t1553_id)
# t429_id = 'fde31f9a-f736-4942-ac4d-24ec5fa05ba7'
# translated_kind.delete_instance(t429_id)
# delete_by_kind_id(t429_id)